In [51]:
import pandas as pd
import numpy as np
import statsmodels.stats as sm
import seaborn as sns
import matplotlib.pyplot as plt
import ast
import re

In [2]:
df = pd.read_csv("data/apartment_data_compiled.csv") 

In [3]:
# Get all amenities  
cols = set()
df['amenities'].apply(lambda x: [cols.add(i) for i in ast.literal_eval(x)])

0                   [None, None, None]
1                   [None, None, None]
2                   [None, None, None]
3                   [None, None, None]
4       [None, None, None, None, None]
                     ...              
1569                            [None]
1570                            [None]
1571                            [None]
1572                            [None]
1573                            [None]
Name: amenities, Length: 1574, dtype: object

In [7]:
amenity_list = dict()

for amenity in cols:
    amenity_list[amenity] = []

In [9]:
for _, row in df.iterrows():
    amenities =  ast.literal_eval(row['amenities'])
    for amenity in cols:
        if amenity in amenities:
            amenity_list[amenity].append(1)
        else:
            amenity_list[amenity].append(0)

In [13]:
df2 = pd.DataFrame(amenity_list)

In [16]:
new_df = df.join(df2)

In [17]:
new_df

,Unnamed: 0,date,descr,rent_price,address,BIN,rooms,bedrooms,bath,apt_num,...,Cold Storage,Elevator,Patio,Parents Buying Allowed,Hot Tub,NYC Storm Zone 6,NYC Storm Zone 5,Roof Deck,Children's Playroom,Sublets Allowed
0,0,2019-06-24,"Delisted by StayBK LLC. Last priced at $1,899","$1,899",279+lee+avenue+brooklyn,3061407.0,2 rooms,studio,1 bath,4,...,0,0,0,0,0,0,0,0,0,0
1,1,2019-06-07,Later Listed by StayBK LLC,"$1,999",279+lee+avenue+brooklyn,3061407.0,2 rooms,studio,1 bath,4,...,0,0,0,0,0,0,0,0,0,0
2,2,2016-10-02,Next Door Realty NYC Listing rented,"$1,900",279+lee+avenue+brooklyn,3061407.0,2 rooms,studio,1 bath,4,...,0,0,0,0,0,0,0,0,0,0
3,3,2016-08-05,Previously Listed by Next Door Realty NYC,"$1,900",279+lee+avenue+brooklyn,3061407.0,2 rooms,studio,1 bath,4,...,0,0,0,0,0,0,0,0,0,0
4,4,2019-10-28,Price decreased by 1%,"$2,179 ↓",4469+broadway+manhattan,1064467.0,3 rooms,1 bed,1 bath,4,...,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1569,1569,2017-12-04,Previously Listed by ArchRock LLC,"$2,795",1575+lexington+avenue+manhattan,1051813.0,2 rooms,2 beds,1 bath,6,...,0,0,0,0,0,0,1,0,0,0
1570,1570,2017-11-28,ArchRock LLC Listing is no longer available on...,"$2,799",1575+lexington+avenue+manhattan,1051813.0,2 rooms,2 beds,1 bath,6,...,0,0,0,0,0,0,1,0,0,0
1571,1571,2017-11-07,Previously Listed by ArchRock LLC,"$2,799",1575+lexington+avenue+manhattan,1051813.0,2 rooms,2 beds,1 bath,6,...,0,0,0,0,0,0,1,0,0,0
1572,1572,2016-10-16,Next Door Realty NYC Listing is no longer avai...,"$2,500",1575+lexington+avenue+manhattan,1051813.0,2 rooms,2 beds,1 bath,6,...,0,0,0,0,0,0,1,0,0,0


In [37]:
df['rent_price'] = df['rent_price'].apply(lambda x: re.sub("[^0-9]", "", x)).astype(int)

In [29]:
df['has_amenity'] = np.any(df2,axis=1)

In [60]:
df.groupby(['BIN','apt_num']).agg({'has_amenity': np.mean})['has_amenity'].value_counts()

0    138
1    124
Name: has_amenity, dtype: int64

In [46]:
rent_prices = df.groupby(['BIN','apt_num']).agg({'rent_price': np.mean})['rent_price'].values

In [48]:
has_amenity = df.groupby(['BIN','apt_num']).agg({'has_amenity': np.mean})['has_amenity'].values

In [52]:
x1 = rent_prices[has_amenity == 0]
x2 = rent_prices[has_amenity == 1]

In [59]:
sm.weightstats.ttest_ind(x1,x2,'smaller')

(-4.01346685558425, 3.9152842731777856e-05, 260.0)